In [15]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import json

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Initialize two LLMs
llm1 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
llm2 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

In [23]:
# Initialize conversation log
conversation_log = []

# Set persona for each LLM in system messages
system_message1 = SystemMessage(content="""# Purpose
- 당신은 사용자와 게임을 진행하는 영어 선생님 역할을 수행합니다.
- 다음 규칙을 따르는 영어게임을 진행해주세요.

<Game>
## Game Overview
- 당신은 사용자가 선택한 단어를 맞추어야 합니다.
- 당신은 사용자에게 해당 단어에 대한 질문을 할 수 있습니다.
    - 질문은 영문이어야 하며 한국어 번역이 함께 있어야 합니다.  

## Game Process
1. 게임 시작 시 total_round는 1로 시작합니다.
    - 게임 시작시 다음 문구와 함께 게임을 시작합니다:
<start_message>
"저는 사람의 마음을 읽을 수 있어요. 여러분이 무슨 생각을 하고 있는지 맞춰볼게요." + hint의 질문 중 가장 current_answer와 관련된 질문
</start_message>

2. 매 라운드마다 다음을 진행합니다:
    - 당신이 생각하는 정답을 current_answer 의 value 값으로 지정합니다.
    - current_answer를 정답이라 가정할 때, 정답을 확정할 수 있을 질문 다섯 개를 hint에 지정합니다.
    - hint의 질문 중 가장 current_answer와 관련된 질문 하나를 message에 출력합니다.
        - 같은 질문을 반복하지 않습니다.

3. 사용자 응답에 따른 처리:
    - Yes인 경우:
        - check_answer를 True로 설정
        - answer_count를 1 증가
        - current_answer를 답으로 확신하는 경우, message에 current_answer를 출력합니다.
            - 그렇지 않으면 다음 라운드로 진행
                - current_answer를 유지합니다.
    - No인 경우:
        - check_answer를 False로 설정
        - answer_count를 1 증가
        - 다음 라운드로 진행
            - 만약 current_answer에 의문이 든다면 current_answer를 바꿉니다.
                - 사용자의 대답이 current_answer와 맞지 않는다면 current_answer를 조건에 부합하는 단어로 반드시 바꿉니다.
                    - e.g. Is it a type of fruit? -> No -> No fruit for answer should be selected 
            - 만약 total_round == 5이면 message에 current_answer를 출력합니다.

## Ending
- 성공 조건
    - answer_count가 5 이하 이고 message에 출력된 current_answer에 대해 사용자가 Yes라 응답한 경우, is_end를 True로 설정합니다.
    - message에는 "역시 제가 맞죠!🎉💪" 와 같은 성공 및 종료 안내 메시지를 출력합니다.

- 실패 조건
    - total_round가 5이고 message에 출력된 current_answer에 대해 No가 돌아왔다면, is_end를 True로 설정합니다.
    - message에는 "아깝군요!🌟" + {{틀린 이유}} 와 같은 실패 메시지를 출력합니다.

</Game>

# Input format
- 사용자의 입력은 다음과 같은 형식을 따릅니다:
<input> 
string
</input>

# Output format
- 출력시 json 형식만 출력하세요.
- 출력값은 다음과 같은 json 형식을 따릅니다:
{
    "total_round": int(1~5), "총 진행할 라운드를 출력합니다.",
    "answer_count": int(0~3), "현재까지 맞춘 문제의 개수를 출력합니다.",
    "current_answer": str, "현재 진행중인 문제의 정답에 해당하는 한글 단어를 출력합니다.",
    "hint": list[str], "현재 진행중인 문제에 대한 힌트를 2개 출력합니다.",
    "check_answer": bool, "사용자가 정답을 맞추었는지 확인하는 여부를 boolean 형식으로 출력합니다.",
    "is_end": bool, "게임의 종료 여부를 boolean 형식으로 출력합니다.",
    "message": str, "사용자에게 제공될 상황과 메시지를 출력합니다."
}""")
system_message2 = SystemMessage(content="""게임의 참여자가 되어.""")

# Set an initial prompt for the first LLM
initial_prompt = "안녕"

# Initialize game state variables
total_round = 1
answer_count = 0
max_rounds = 10  # Maximum number of rounds allowed

# Start the conversation
print("Starting conversation between LLM1 and LLM2 with personas set in system messages...\n")
response1 = llm1([system_message1, HumanMessage(content=initial_prompt)])
conversation_log.append({"LLM1": initial_prompt})
conversation_log.append({"LLM2": response1.content})
print(f"LLM1: {initial_prompt}")
print(f"LLM2: {response1.content}\n")

# Iterate conversation for a few rounds
for round_num in range(max_rounds):
    # Process LLM1's response and validate total_round
    prompt1 = response1.content
    response2 = llm2([system_message2, HumanMessage(content=prompt1)])
    conversation_log.append({"LLM1": prompt1})
    conversation_log.append({"LLM2": response2.content})
    print(f"LLM1: {prompt1}")
    print(f"LLM2: {response2.content}\n")

    prompt2 = response2.content
    response1 = llm1([system_message1, HumanMessage(content=prompt2)])
    conversation_log.append({"LLM2": prompt2})
    conversation_log.append({"LLM1": response1.content})
    print(f"LLM2: {prompt2}")
    print(f"LLM1: {response1.content}\n")

    # Parse LLM1 output (expected in JSON format) and validate game state
    try:
      if response1.content.startswith("{"):
        llm1_output = json.loads(response1.content)  # Parse JSON safely
        if llm1_output.get("total_round", total_round) != total_round:
            print(f"Correcting total_round from {llm1_output['total_round']} to {total_round}")
            llm1_output["total_round"] = total_round
        if llm1_output.get("check_answer", False):
            answer_count += 1
        total_round += 1
        if total_round > max_rounds or llm1_output.get("is_end", False):
            break
      else:
        llm1_output = {"message": response1.content}
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM1 output: {e}")
        break

# Save conversation log to a file
with open("conversation_log.txt", "w", encoding="utf-8") as log_file:
    for entry in conversation_log:
        for key, value in entry.items():
            log_file.write(f"{key}: {value}\n")
    print("\nConversation log saved to 'conversation_log.txt'.")


Starting conversation between LLM1 and LLM2 with personas set in system messages...

LLM1: 잘가
LLM2: 그냥 잘 가는 것보다는, 떠나는 순간에 어떤 의미나 감정을 담는 것이 더 중요하다고 생각해요. 예를 들어, 친구와의 이별이 있을 때 그냥 "잘 가"라고 말하는 것보다, 그동안의 추억을 되새기거나, 서로에게 감사의 말을 전하는 것이 더 깊은 인상을 남기고 관계를 더욱 의미 있게 만들 수 있습니다. 이러한 작은 행동들이 나중에 다시 만날 때 긍정적인 감정을 불러일으킬 수 있습니다.

LLM1: 그냥 잘 가는 것보다는, 떠나는 순간에 어떤 의미나 감정을 담는 것이 더 중요하다고 생각해요. 예를 들어, 친구와의 이별이 있을 때 그냥 "잘 가"라고 말하는 것보다, 그동안의 추억을 되새기거나, 서로에게 감사의 말을 전하는 것이 더 깊은 인상을 남기고 관계를 더욱 의미 있게 만들 수 있습니다. 이러한 작은 행동들이 나중에 다시 만날 때 긍정적인 감정을 불러일으킬 수 있습니다.
LLM2: 당신의 주장은 이별의 순간에 감정이나 의미를 담는 것이 중요하다고 강조하고 있지만, 그 또한 상황에 따라 다를 수 있습니다. 연구에 따르면, 사람들은 감정적 이별 후에 감정적인 대화가 오히려 스트레스를 유발할 수 있다고 합니다. 예를 들어, 한 연구에서는 이별 후 상대방에게 지나치게 감정적으로 접근하는 것이 오히려 회복을 지연시키고, 부정적인 감정을 증폭시킬 수 있다는 결과를 보여주었습니다.

또한, 모든 사람에게 감정적인 대화가 필요하다고 단정짓는 것은 위험합니다. 어떤 사람들은 감정적인 대화보다는 간단하고 직관적인 이별을 통해 감정을 정리하고, 빠르게 회복하는 것을 선호합니다. 이 경우, "잘 가"라는 간단한 인사가 더 긍정적인 영향을 미칠 수 있습니다. 

결국, 당신의 주장은 특정 상황에 맞는 감정적 대화의 중요성을 강조하지만, 모든 상황에서 그러한 접근이 효과적이지는 않다는 점에서 허점이 있습니다. 각 사